# Module 2 Homework

## Import packages and set constants

In [1]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

import requests

### Question 1: IPO Filings Web Scraping and Data Processing

**What's the total sum ($m) of 2023 filings that happened on Fridays?**

In [11]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipos_filings = pd.read_html(response.text)

ipos_filings = ipos_filings[0]
ipos_filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     330 non-null    object
 1   Symbol          330 non-null    object
 2   Company Name    330 non-null    object
 3   Price Range     330 non-null    object
 4   Shares Offered  330 non-null    object
dtypes: object(5)
memory usage: 13.0+ KB


/tmp/ipykernel_672/2727273146.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipos_filings = pd.read_html(response.text)


In [12]:
# Convert to datetime
ipos_filings["Filing Date"] = pd.to_datetime(ipos_filings["Filing Date"], format="%b %d, %Y")

In [19]:
data_types = {
    "Filing Date": "datetime64[ns]",
    "Symbol": "string",
    "Company Name": "string",
    "Price Range": "string",
    "Shares Offered": "float64"
}

ipos_filings["Shares Offered"] = ipos_filings["Shares Offered"].replace("-", np.NaN)

ipos_filings = ipos_filings.astype(data_types)

In [20]:
ipos_filings.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-14,ICON,Icon Energy Corp.,$4.00 - $6.00,1250000.0
1,2024-05-13,FCHS,"First Choice Healthcare Solutions, Inc.",$5.00,1200000.0
2,2024-05-13,KTA,Kingtitan Technology Limited,$4.00 - $6.00,3750000.0
3,2024-05-13,GALE,Galleinphi Inc.,$4.00 - $6.00,3000000.0
4,2024-05-13,CHEB,Chenghe Acquisition II Co.,$10.00,7500000.0


In [74]:
import re

def get_avg_price(input_string):
    matches = re.findall(r'(\d+.\d+)', input_string)
    avg_price = np.NaN
    if len(matches) > 1:
        avg_price = (float(matches[0]) + float(matches[1])) / 2
    elif len(matches) == 1:
        avg_price = float(matches[0])

    return avg_price


In [77]:
ipos_filings["Avg_price"] = ipos_filings["Price Range"].apply(lambda x: get_avg_price(x))

In [80]:
ipos_filings["Shares_offered_value"] = ipos_filings["Shares Offered"] * ipos_filings["Avg_price"]

In [85]:
ipos_filings["Day_of_week"] = ipos_filings["Filing Date"].dt.dayofweek
ipos_filings["Year"] = ipos_filings["Filing Date"].dt.year

In [91]:
ipos_filings.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg Price,Avg_price,Shares_offered_value,Day_of_week,Year
0,2024-05-14,ICON,Icon Energy Corp.,$4.00 - $6.00,1250000.0,5.0,5.0,6250000.0,1,2024
1,2024-05-13,FCHS,"First Choice Healthcare Solutions, Inc.",$5.00,1200000.0,5.0,5.0,6000000.0,0,2024
2,2024-05-13,KTA,Kingtitan Technology Limited,$4.00 - $6.00,3750000.0,5.0,5.0,18750000.0,0,2024
3,2024-05-13,GALE,Galleinphi Inc.,$4.00 - $6.00,3000000.0,5.0,5.0,15000000.0,0,2024
4,2024-05-13,CHEB,Chenghe Acquisition II Co.,$10.00,7500000.0,10.0,10.0,75000000.0,0,2024


In [90]:
ipos_filings[(ipos_filings["Year"]==2023) & (ipos_filings["Day_of_week"]==4)]["Shares_offered_value"].sum()

281325000.0